<a href="https://colab.research.google.com/github/Srinidhi-15/MyProjects/blob/main/credit_system_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix, precision_score,recall_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils import resample

In [2]:
train_data = pd.read_csv("/content/fraudTrain.csv")
test_data = pd.read_csv("/content/fraudTest.csv")

In [3]:
train_data.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1.325376e+09,36.011293,-82.048315,0.0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1.325376e+09,49.159047,-118.186462,0.0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1.325376e+09,43.150704,-112.154481,0.0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1.325376e+09,47.034331,-112.561071,0.0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1.325376e+09,38.674999,-78.632459,0.0


In [4]:
test_data.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1.371817e+09,33.986391,-81.200714,0.0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1.371817e+09,39.450498,-109.960431,0.0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1.371817e+09,40.495810,-74.196111,0.0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1.371817e+09,28.812398,-80.883061,0.0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1.371817e+09,44.959148,-85.884734,0.0


In [5]:
test_data = test_data[["trans_date_trans_time","cc_num","amt","gender","zip","lat","long","city_pop","merch_lat","merch_long","is_fraud"]]
train_data = train_data[["trans_date_trans_time","cc_num","amt","gender","zip","lat","long","city_pop","merch_lat","merch_long","is_fraud"]]

In [6]:
train_data.is_fraud.value_counts()


0.0    11657
1.0       48
Name: is_fraud, dtype: int64

In [7]:
majority = train_data[train_data["is_fraud"]==0]
minority = train_data[train_data["is_fraud"]==1]
upsample = resample(minority,replace=True,n_samples=len(majority))
train_data = pd.concat([majority,upsample])

In [8]:
train_data["trans_date_trans_time"] = pd.to_datetime(train_data["trans_date_trans_time"])
test_data["trans_date_trans_time"] = pd.to_datetime(test_data["trans_date_trans_time"])

train_data['hour'] = train_data['trans_date_trans_time'].dt.hour
test_data['hour'] = test_data['trans_date_trans_time'].dt.hour

train_data['day'] = train_data['trans_date_trans_time'].dt.day
test_data['day'] = test_data['trans_date_trans_time'].dt.day

train_data['month'] = train_data['trans_date_trans_time'].dt.month
test_data['month'] = test_data['trans_date_trans_time'].dt.month

In [9]:
train_data.drop("trans_date_trans_time",axis=1,inplace=True)
test_data.drop("trans_date_trans_time",axis=1,inplace=True)

In [10]:
train_data.head()

,cc_num,amt,gender,zip,lat,long,city_pop,merch_lat,merch_long,is_fraud,hour,day,month
0,2703186189652095,4.97,F,28654,36.0788,-81.1781,3495,36.011293,-82.048315,0.0,0,1,1
1,630423337322,107.23,F,99160,48.8878,-118.2105,149,49.159047,-118.186462,0.0,0,1,1
2,38859492057661,220.11,M,83252,42.1808,-112.2620,4154,43.150704,-112.154481,0.0,0,1,1
3,3534093764340240,45.00,M,59632,46.2306,-112.1138,1939,47.034331,-112.561071,0.0,0,1,1
4,375534208663984,41.96,M,24433,38.4207,-79.4629,99,38.674999,-78.632459,0.0,0,1,1


In [11]:
train_data['gender'] = train_data['gender'].map({'F':0,'M':1})
test_data['gender'] = test_data['gender'].map({'F':0,'M':1})

In [12]:
train_data.head()

,cc_num,amt,gender,zip,lat,long,city_pop,merch_lat,merch_long,is_fraud,hour,day,month
0,2703186189652095,4.97,0,28654,36.0788,-81.1781,3495,36.011293,-82.048315,0.0,0,1,1
1,630423337322,107.23,0,99160,48.8878,-118.2105,149,49.159047,-118.186462,0.0,0,1,1
2,38859492057661,220.11,1,83252,42.1808,-112.2620,4154,43.150704,-112.154481,0.0,0,1,1
3,3534093764340240,45.00,1,59632,46.2306,-112.1138,1939,47.034331,-112.561071,0.0,0,1,1
4,375534208663984,41.96,1,24433,38.4207,-79.4629,99,38.674999,-78.632459,0.0,0,1,1


In [13]:
train_data = train_data.dropna()
test_data = test_data.dropna()

In [14]:
X_train = train_data.drop('is_fraud',axis=1)
X_test = test_data.drop('is_fraud',axis=1)

y_train = train_data['is_fraud']
y_test = test_data['is_fraud']

In [15]:
scale = StandardScaler()
X_train = scale.fit_transform(X_train)
X_test = scale.transform(X_test)

In [16]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

LogisticRegression()

In [17]:
train_accuracy = lr.score(X_train, y_train)

test_accuracy = lr.score(X_test, y_test)

print("Accuracy on Training Set:", train_accuracy)
print("Accuracy on Test Set:", test_accuracy)


Accuracy on Training Set: 0.9453547224843442
Accuracy on Test Set: 0.989098371168398


In [18]:

y_pred = lr.predict(X_test)


cm = confusion_matrix(y_test, y_pred)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = 2 * (precision * recall) / (precision + recall)


class_report = classification_report(y_test, y_pred)

accuracy_metrics = np.array([
    ["Accuracy", (cm[0, 0] + cm[1, 1]) / np.sum(cm)],
    ["Precision", precision],
    ["Recall", recall],
    ["F1-Score", f1]
])

print("Confusion Matrix:")
print(cm)
print("\nClassification Report:")
print(class_report)
print("\nAccuracy and Other Metrics:")
print(accuracy_metrics)


Confusion Matrix:
[[7700   75]
 [  10   12]]

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      0.99      0.99      7775
         1.0       0.14      0.55      0.22        22

    accuracy                           0.99      7797
   macro avg       0.57      0.77      0.61      7797
weighted avg       1.00      0.99      0.99      7797


Accuracy and Other Metrics:
[['Accuracy' '0.989098371168398']
 ['Precision' '0.13793103448275862']
 ['Recall' '0.5454545454545454']
 ['F1-Score' '0.22018348623853207']]


In [19]:
def get_user_input():
    cc_num = float(input("Enter credit card number: "))
    amt = float(input("Enter transaction amount: "))
    gender = input("Enter gender (F/M): ")
    zip_code = int(input("Enter ZIP code: "))
    lat = float(input("Enter latitude: "))
    long = float(input("Enter longitude: "))
    city_pop = int(input("Enter city population: "))
    merch_lat = float(input("Enter merchant latitude: "))
    merch_long = float(input("Enter merchant longitude: "))
    hour = int(input("Enter hour of the day: "))
    day = int(input("Enter day of the month: "))
    month = int(input("Enter month of the year: "))


    input_data = pd.DataFrame({
        'cc_num': [cc_num],
        'amt': [amt],
        'gender': [gender],
        'zip': [zip_code],
        'lat': [lat],
        'long': [long],
        'city_pop': [city_pop],
        'merch_lat': [merch_lat],
        'merch_long': [merch_long],
        'hour': [hour],
        'day': [day],
        'month': [month]
    })

    return input_data


In [20]:
new_data = get_user_input()


new_data['gender'] = new_data['gender'].map({'F': 0, 'M': 1})


new_data_scaled = scale.transform(new_data)


prediction = lr.predict(new_data_scaled)


print("Predicted Fraud (1) or Not Fraud (0):", prediction[0])

Enter credit card number: 630423337322
Enter transaction amount: 60000
Enter gender (F/M): F
Enter ZIP code: 606003
Enter latitude: 56
Enter longitude: 68
Enter city population: 50000
Enter merchant latitude: 70
Enter merchant longitude: 67
Enter hour of the day: 12
Enter day of the month: 14
Enter month of the year: 7
Predicted Fraud (1) or Not Fraud (0): 1.0


In [23]:
new_data = get_user_input()


new_data['gender'] = new_data['gender'].map({'F': 0, 'M': 1})


new_data_scaled = scale.transform(new_data)


prediction = lr.predict(new_data_scaled)


print("Predicted Fraud (1) or Not Fraud (0):", prediction[0])

Enter credit card number: 630423337322
Enter transaction amount: 107.23
Enter gender (F/M): m
Enter ZIP code: 99160
Enter latitude: 48.88
Enter longitude: -118.21
Enter city population: 149
Enter merchant latitude: 49.15
Enter merchant longitude: -118.18
Enter hour of the day: 1
Enter day of the month: 1
Enter month of the year: 1


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values